<a href="https://colab.research.google.com/github/Atharva-Peshkar/DeepFake-Detector/blob/main/DeepFake_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.layers import TimeDistributed, Dense, LSTM, Dropout, Input, Flatten, Reshape, GlobalAveragePooling2D
from keras.models import Model, Sequential
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Input
from keras.layers import Flatten
from keras.layers import Reshape
from keras.layers import GlobalAveragePooling2D


base_model = InceptionV3(input_shape = (299,299, 3),include_top=False, weights='imagenet')

for layer_model in base_model.layers:
  layer_model.trainable = False


crnn = Sequential([
                   TimeDistributed(base_model,input_shape=(42,229,229,3)),   #Input to InceptionV3 wrapped in TimeDistributed Layer = (Timesteps,Height,Width,Chanels)
                   TimeDistributed(Flatten()),     #Output from InceptionV3 (Batch_Size,8,8,2048) passed through this to remove the spacial data and only keep the temporal data
                   #Flatten(),
                   LSTM(2048,dropout=0.5,return_sequences=False),  #Input Shape = (Timesteps,Features)
                   Dense(512),
                   Dropout(0.5),
                   Dense(1,activation="sigmoid")])

87916544/87910968 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 42, 8, 8, 2048)    21802784  
_________________________________________________________________
time_distributed_1 (TimeDist (None, 42, 131072)        0         
_________________________________________________________________
lstm (LSTM)                  (None, 256)               134480896 
_________________________________________________________________
dense (Dense)                (None, 64)                16448     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 195       
Total params: 156,300,323
Trainable params: 134,497,539
N

In [ ]:
crnn.compile('adam', loss='categorical_crossentropy')

In [ ]:
from keras.optimizers import Adam

crnn.compile(optimizer = Adam(lr=0.00001, decay=0.000001),
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'],run_eagerly=False)

#print(base_model.summary())

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
from tqdm.notebook import tqdm
import cv2
from keras.preprocessing import image
import matplotlib.pyplot as plt

#Define paths to the directories holding the training images.
train_data_dir_pristine = '/gdrive/My Drive/Colab Notebooks/DeepFake/Test_Model_Training_Data/Pristine/'
train_data_dir_fake = '/gdrive/My Drive/Colab Notebooks/DeepFake/Test_Model_Training_Data/Deepfake/'

# Get a list of all the image names in the directories. Eg: Frame1.jpg, Frame2.jpg etc.
pristine_data = os.listdir(train_data_dir_pristine)[0:21]
deepfake_data = os.listdir(train_data_dir_fake)[0:21]

training_data_X = []
for every_img in tqdm(pristine_data):
  '''img_pris_load = image.load_img(train_data_dir_pristine + every_img, target_size=(299,299,3)) #Load image from dir
  img_to_arr_pris = image.img_to_array(img_pris_load) #Convert the image to np array.
  img_to_arr_pris = np.expand_dims(img_to_arr_pris, axis=0) #Add a dimension to the array -  the image index. (Image index, height, width, channels)
  training_data_X = np.vstack([img_to_arr_pris]) #Stack the image arrays vertically.'''
  image = cv2.imread (train_data_dir_pristine+every_img)
  training_data_X.append (image)


  #rg = plt.imshow(image.array_to_img(training_data_X_Pristine[0]))

training_data_Y_Pristine = np.full((len(pristine_data),1),'Pristine')   # Create an array with labels for each image in the X array.

# Everything same as above, except this is for Deepfakes.
for every_img in tqdm(deepfake_data):
  '''img_fake_load = image.load_img(train_data_dir_fake + every_img, target_size=(299,299,3))
  img_to_arr_fake = image.img_to_array(img_fake_load)
  img_to_arr_fake = np.expand_dims(img_to_arr_fake, axis=0)
  training_data_X = np.vstack([img_to_arr_fake])'''
  image = cv2.imread (train_data_dir_fake+every_img)
  training_data_X.append (image)
  #rg = plt.imshow(image.array_to_img(training_data_X_DeepFake[0]))

training_data_Y_DeepFake = np.full((len(deepfake_data),1),'Deepfake')

# Combine the Pristine and Deepfake arrays together by stacking both X and Y arrays.
#training_data_X = np.concatenate((training_data_X_Pristine,training_data_X_DeepFake))
training_data_Y = np.vstack([training_data_Y_Pristine,training_data_Y_DeepFake])

#print(training_data_X.shape)
print('X_data shape:', np.array(training_data_X).shape)
print(training_data_Y.shape)
# Add our data-augmentation parameters to ImageDataGenerator
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
training_data_Y = lb.fit_transform(training_data_Y)


X_data shape: (42, 299, 299, 3)
(42, 1)


In [ ]:
history = model.fit(
            np.array(training_data_X),
            np.array(training_data_Y),
            steps_per_epoch = 10,
            epochs = 100,
            verbose = 1)